# ETL & Data Preparation : Extract, Transform, Load

## Objectives

* To prepare a unified, analysis-ready dataset that supports examining how renewable energy deployment and energy efficiency impact CO₂ emissions over time, and to detect structural tipping points in emission trends. The dataset enables hypothesis testing, machine learning, and interactive dashboard visualisation to guide policy and research.

## Inputs

- **Global Sustainable Energy (Kaggle)**  
  Country-level annual indicators on electricity generation, renewable energy share, and CO₂ emissions (2000–2020).

- **World Bank Population (SP.POP.TOTL)**  
  Total population data by country (1960–2023), subset for 2000–2020 to align with energy data coverage.

- **UNSD M49 Region Mapping**  
  Static country-to-region classification to support regional comparisons and aggregation.

## Outputs

* A cleaned, merged dataset containing harmonised country-year records (2000–2020), enriched with population and regional classifications.  
This integrated dataset forms the foundation for statistical analysis, hypothesis validation, and predictive modelling in a Streamlit dashboard.

---

### Load and Inspect the Data
Understand the raw data — check columns, types, shape, duplicates and any obvious issues

In [1]:
# Import required libraries and set up output directory
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from seaborn import boxplot
import plotly.express as px
import os

In [2]:
# Create folder to save images 
os.makedirs("images", exist_ok=True)

### Extract Datasets

Step 1: Load Datasets

In [ ]:
# Load the global energy dataset
df_energy = pd.read_csv("../data/raw/global-data-on-sustainable-energy.csv")

# Load the population dataset
df_region = pd.read_csv("../data/raw/unsd_country_region_mapping.csv", sep=";")


df_population = pd.read_csv("../data/raw/world_bank_population.csv")



Step 2: Preview the Raw Data

In [27]:
from IPython.display import display

# Use display() to preview the first few rows of each dataset
display(df_energy.head())
display(df_population.head())
display(df_region.head())


,Entity,Year,Access to electricity (% of population),Access to clean fuels for cooking,Renewable-electricity-generating-capacity-per-capita,Financial flows to developing countries (US $),Renewable energy share in the total final energy consumption (%),Electricity from fossil fuels (TWh),Electricity from nuclear (TWh),Electricity from renewables (TWh),...,Primary energy consumption per capita (kWh/person),Energy intensity level of primary energy (MJ/$2017 PPP GDP),Value_co2_emissions_kt_by_country,Renewables (% equivalent primary energy),gdp_growth,gdp_per_capita,Density\n(P/Km2),Land Area(Km2),Latitude,Longitude
0,Afghanistan,2000,1.613591,6.2,9.22,20000.0,44.99,0.16,0.0,0.31,...,302.59482,1.64,760.000000,NaN,NaN,NaN,60,652230.0,33.93911,67.709953
1,Afghanistan,2001,4.074574,7.2,8.86,130000.0,45.60,0.09,0.0,0.50,...,236.89185,1.74,730.000000,NaN,NaN,NaN,60,652230.0,33.93911,67.709953
2,Afghanistan,2002,9.409158,8.2,8.47,3950000.0,37.83,0.13,0.0,0.56,...,210.86215,1.40,1029.999971,NaN,NaN,179.426579,60,652230.0,33.93911,67.709953
3,Afghanistan,2003,14.738506,9.5,8.09,25970000.0,36.66,0.31,0.0,0.63,...,229.96822,1.40,1220.000029,NaN,8.832278,190.683814,60,652230.0,33.93911,67.709953
4,Afghanistan,2004,20.064968,10.9,7.75,NaN,44.24,0.33,0.0,0.56,...,204.23125,1.20,1029.999971,NaN,1.414118,211.382074,60,652230.0,33.93911,67.709953


,STRUCTURE,STRUCTURE_ID,ACTION,FREQ_ID,FREQ_NAME,REF_AREA_ID,REF_AREA_NAME,INDICATOR_ID,INDICATOR_NAME,SEX_ID,...,DATA_SOURCE_NAME,UNIT_TYPE_ID,UNIT_TYPE_NAME,TIME_FORMAT_ID,TIME_FORMAT_NAME,COMMENT_OBS,OBS_STATUS_ID,OBS_STATUS_NAME,OBS_CONF_ID,OBS_CONF_NAME
0,datastructure,WB.DATA360:DS_DATA360(1.2),I,A,Annual,AFE,Africa Eastern and Southern,WB_WDI_SP_POP_TOTL,"Population, total",_T,...,World Development Indicators (WDI),COUNT,Count (Integer),P1Y,Annual,NaN,A,Normal value,PU,Public
1,datastructure,WB.DATA360:DS_DATA360(1.2),I,A,Annual,AFW,Africa Western and Central,WB_WDI_SP_POP_TOTL,"Population, total",_T,...,World Development Indicators (WDI),COUNT,Count (Integer),P1Y,Annual,NaN,A,Normal value,PU,Public
2,datastructure,WB.DATA360:DS_DATA360(1.2),I,A,Annual,ARB,Arab World,WB_WDI_SP_POP_TOTL,"Population, total",_T,...,World Development Indicators (WDI),COUNT,Count (Integer),P1Y,Annual,NaN,A,Normal value,PU,Public
3,datastructure,WB.DATA360:DS_DATA360(1.2),I,A,Annual,CSS,Caribbean small states,WB_WDI_SP_POP_TOTL,"Population, total",_T,...,World Development Indicators (WDI),COUNT,Count (Integer),P1Y,Annual,NaN,A,Normal value,PU,Public
4,datastructure,WB.DATA360:DS_DATA360(1.2),I,A,Annual,CEB,Central Electricity Board (CEB),WB_WDI_SP_POP_TOTL,"Population, total",_T,...,World Development Indicators (WDI),COUNT,Count (Integer),P1Y,Annual,NaN,A,Normal value,PU,Public


,Global Code,Global Name,Region Code,Region Name,Sub-region Code,Sub-region Name,Intermediate Region Code,Intermediate Region Name,Country or Area,M49 Code,ISO-alpha2 Code,ISO-alpha3 Code,Least Developed Countries (LDC),Land Locked Developing Countries (LLDC),"Small Island Developing States (SIDS),"
0,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Algeria,12,DZ,DZA,NaN,NaN,","
1,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Egypt,818,EG,EGY,NaN,NaN,","
2,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Libya,434,LY,LBY,NaN,NaN,","
3,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Morocco,504,MA,MAR,NaN,NaN,","
4,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Sudan,729,SD,SDN,x,NaN,","


Step 3: Inspect Dataset Structure

In [28]:
# Inspect the structure and nulls in the energy dataset
print("df_energy:")
print(df_energy.shape)
df_energy.info()
print(df_energy.isnull().sum())

df_energy:
(3649, 21)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3649 entries, 0 to 3648
Data columns (total 21 columns):
 #   Column                                                            Non-Null Count  Dtype  
---  ------                                                            --------------  -----  
 0   Entity                                                            3649 non-null   object 
 1   Year                                                              3649 non-null   int64  
 2   Access to electricity (% of population)                           3639 non-null   float64
 3   Access to clean fuels for cooking                                 3480 non-null   float64
 4   Renewable-electricity-generating-capacity-per-capita              2718 non-null   float64
 5   Financial flows to developing countries (US $)                    1560 non-null   float64
 6   Renewable energy share in the total final energy consumption (%)  3455 non-null   float64
 7   Electricity

In [29]:
# Inspect the structure and nulls in the population dataset
print("\ndf_population:")
print(df_population.shape)
df_population.info()
print(df_population.isnull().sum())


df_population:
(16930, 45)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16930 entries, 0 to 16929
Data columns (total 45 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   STRUCTURE              16930 non-null  object 
 1   STRUCTURE_ID           16930 non-null  object 
 2   ACTION                 16930 non-null  object 
 3   FREQ_ID                16930 non-null  object 
 4   FREQ_NAME              16930 non-null  object 
 5   REF_AREA_ID            16930 non-null  object 
 6   REF_AREA_NAME          16930 non-null  object 
 7   INDICATOR_ID           16930 non-null  object 
 8   INDICATOR_NAME         16930 non-null  object 
 9   SEX_ID                 16930 non-null  object 
 10  SEX_NAME               16930 non-null  object 
 11  AGE_ID                 16930 non-null  object 
 12  AGE_NAME               16930 non-null  object 
 13  URBANISATION_ID        16930 non-null  object 
 14  URBANISATION_NAME      169

In [30]:
# Inspect the structure and nulls in the region mapping dataset
print("\ndf_region:")
print(df_region.shape)
df_region.info()
print(df_region.isnull().sum())


df_region:
(248, 15)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 15 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Global Code                              248 non-null    int64  
 1   Global Name                              248 non-null    object 
 2   Region Code                              247 non-null    float64
 3   Region Name                              247 non-null    object 
 4   Sub-region Code                          247 non-null    float64
 5   Sub-region Name                          247 non-null    object 
 6   Intermediate Region Code                 105 non-null    float64
 7   Intermediate Region Name                 105 non-null    object 
 8   Country or Area                          248 non-null    object 
 9   M49 Code                                 248 non-null    int64  
 10  ISO-alpha2 Code             

Step 4: Clean Column Names in All Datasets

In [31]:
# Function to clean column names: lowercase, trim spaces, replace internal spaces with underscores, remove special characters
def clean_column_names(df):
    df.columns = (
        df.columns.str.strip()                      # remove leading/trailing whitespace
                  .str.lower()                      # make lowercase
                  .str.replace(r'[^\w\s]', '', regex=True)  # remove special characters
                  .str.replace(r'\s+', '_', regex=True)     # replace space(s) with underscore
    )
    return df

# Apply to all datasets
df_energy = clean_column_names(df_energy)
df_population = clean_column_names(df_population)
df_region = clean_column_names(df_region)


Step 5: Inspect Dataset Structure

In [34]:
# inspect the cleaned structure and nulls in the energy dataset
print("df_energy:")
print(df_energy.shape)
df_energy.info()
print(df_energy.isnull().sum())

df_energy:
(3649, 21)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3649 entries, 0 to 3648
Data columns (total 21 columns):
 #   Column                                                         Non-Null Count  Dtype  
---  ------                                                         --------------  -----  
 0   entity                                                         3649 non-null   object 
 1   year                                                           3649 non-null   int64  
 2   access_to_electricity_of_population                            3639 non-null   float64
 3   access_to_clean_fuels_for_cooking                              3480 non-null   float64
 4   renewableelectricitygeneratingcapacitypercapita                2718 non-null   float64
 5   financial_flows_to_developing_countries_us_                    1560 non-null   float64
 6   renewable_energy_share_in_the_total_final_energy_consumption_  3455 non-null   float64
 7   electricity_from_fossil_fuels_twh     

In [35]:
# inspect the cleaned structure and nulls in the population dataset
print("\ndf_population:")
print(df_population.shape)
df_population.info()
print(df_population.isnull().sum())


df_population:
(16930, 45)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16930 entries, 0 to 16929
Data columns (total 45 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   structure              16930 non-null  object 
 1   structure_id           16930 non-null  object 
 2   action                 16930 non-null  object 
 3   freq_id                16930 non-null  object 
 4   freq_name              16930 non-null  object 
 5   ref_area_id            16930 non-null  object 
 6   ref_area_name          16930 non-null  object 
 7   indicator_id           16930 non-null  object 
 8   indicator_name         16930 non-null  object 
 9   sex_id                 16930 non-null  object 
 10  sex_name               16930 non-null  object 
 11  age_id                 16930 non-null  object 
 12  age_name               16930 non-null  object 
 13  urbanisation_id        16930 non-null  object 
 14  urbanisation_name      169

In [36]:
# inspect the cleaned structure and nulls in the region dataset
print("\ndf_region:")
print(df_region.shape)
df_region.info()
print(df_region.isnull().sum())


df_region:
(248, 15)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 15 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   global_code                            248 non-null    int64  
 1   global_name                            248 non-null    object 
 2   region_code                            247 non-null    float64
 3   region_name                            247 non-null    object 
 4   subregion_code                         247 non-null    float64
 5   subregion_name                         247 non-null    object 
 6   intermediate_region_code               105 non-null    float64
 7   intermediate_region_name               105 non-null    object 
 8   country_or_area                        248 non-null    object 
 9   m49_code                               248 non-null    int64  
 10  isoalpha2_code                         247 non-null 